# Check for both datasets normality

In [ ]:
import os
import numpy as np

# Load the datasets
X_unlabeled1 = np.load('NEW_DATASET/interhand2.6m_keypoints30fps.npy')
X_unlabeled2 = np.load('NEW_DATASET/hsosunlabeled.npy')
X_mediapipe = np.load('NEW_DATASET/dataset_20240709_X_3096.npy')

# Print dataset shapes
print(f"Shape of X_unlabeled1: {X_unlabeled1.shape}")
print(f"Shape of X_unlabeled2: {X_unlabeled2.shape}")
print(f"Shape of X_mediapipe: {X_mediapipe.shape}")

# Function to normalize the dataset element-wise
def normalize_dataset_elementwise(dataset, frame_width=512, frame_height=334):
    """
    Normalize the dataset element-wise:
    - Normalize x (index 0, 3, 6, ...) by frame width.
    - Normalize y (index 1, 4, 7, ...) by frame height.
    - Normalize z (index 2, 5, 8, ...) relative to the wrist and scale by frame width.
    """
    normalized_dataset = np.empty_like(dataset)  # Preserve the original shape
    for i in range(dataset.shape[0]):
        for j in range(dataset.shape[1]):
            frame = dataset[i, j].reshape(-1, 3)  # Reshape into (21, 3) for processing
            wrist_z = frame[0, 2]  # Wrist z-coordinate
            frame[:, 0] /= frame_width  # Normalize x
            frame[:, 1] /= frame_height  # Normalize y
            frame[:, 2] = (frame[:, 2] - wrist_z) / frame_width  # Normalize z
            normalized_dataset[i, j] = frame.flatten()  # Flatten back to original
    return normalized_dataset

# Normalize the dataset
normalized_output_path = "normalized_interhand2.6m.npy"
X_unlabeled1_normalized = normalize_dataset_elementwise(X_unlabeled1)

# Save the normalized dataset
np.save(normalized_output_path, X_unlabeled1_normalized)
print(f"Normalized dataset saved to {normalized_output_path}")

# Reload the normalized dataset to calculate ranges
normalized_dataset = np.load(normalized_output_path)
print(f"Normalized dataset shape: {normalized_dataset.shape}")

# Check data ranges of the normalized dataset
def check_data_ranges(dataset, name):
    """
    Print the min and max values of the dataset.
    """
    min_val = np.min(dataset)
    max_val = np.max(dataset)
    print(f"{name} - Min Value: {min_val}, Max Value: {max_val}")

check_data_ranges(normalized_dataset, "X_unlabeled1_normalized")

# Scale the normalized dataset to match the range of other datasets
def calculate_scaling_factor(dataset1, dataset2, metric="range"):
    """
    Calculate a scaling factor to align the scales of two datasets based on their range or standard deviation.
    """
    if metric == "range":
        range1 = np.max(dataset1) - np.min(dataset1)
        range2 = np.max(dataset2) - np.min(dataset2)
        factor = range2 / range1
    elif metric == "std":
        std1 = np.std(dataset1)
        std2 = np.std(dataset2)
        factor = std2 / std1
    else:
        raise ValueError("Unsupported metric. Use 'range' or 'std'.")
    return factor

scaling_factor = calculate_scaling_factor(normalized_dataset, X_mediapipe, metric="range")
print(f"Scaling factor calculated: {scaling_factor}")

# Apply scaling
scaled_output_path = "scaled_interhand2.6m.npy"
X_unlabeled1_scaled = normalized_dataset * scaling_factor
np.save(scaled_output_path, X_unlabeled1_scaled)

# Check data ranges after scaling
scaled_dataset = np.load(scaled_output_path)
check_data_ranges(scaled_dataset, "X_unlabeled1_scaled")
check_data_ranges(X_unlabeled2, "X_unlabeled2_normalized")
check_data_ranges(X_mediapipe, "X_mediapipe_normalized")

print(f"Final scaled dataset saved to {scaled_output_path}")


In [ ]:
import os
import numpy as np

# Load the normalized datasets
X_unlabeled1_normalized = np.load('NEW_DATASET/scaled_interhand2.6m.npy', mmap_mode='r')
X_unlabeled2_normalized = np.load('NEW_DATASET/hsosunlabeled.npy', mmap_mode='r')
X_mediapipe_normalized = np.load('NEW_DATASET/dataset_20240709_X_3096.npy', mmap_mode='r')

# Function to calculate the scaling factor
def calculate_scaling_factor(dataset1, dataset2, metric="range"):
    """
    Calculate a scaling factor to align the scales of two datasets based on their range or standard deviation.
    Args:
        dataset1: First dataset to be scaled.
        dataset2: Reference dataset.
        metric: "range" or "std" to calculate the factor.
    Returns:
        Scaling factor.
    """
    if metric == "range":
        range1 = np.max(dataset1) - np.min(dataset1)
        range2 = np.max(dataset2) - np.min(dataset2)
        factor = range2 / range1
    elif metric == "std":
        std1 = np.std(dataset1)
        std2 = np.std(dataset2)
        factor = std2 / std1
    else:
        raise ValueError("Unsupported metric. Use 'range' or 'std'.")
    return factor

# Calculate scaling factor based on range (you can switch to "std" if preferred)
scaling_factor = calculate_scaling_factor(X_unlabeled1_normalized, X_mediapipe_normalized, metric="range")
print(f"Scaling factor calculated: {scaling_factor}")

# Apply scaling to X_unlabeled1_normalized
scaled_output_path = "scaled_interhand2.6m.npy"
with open(scaled_output_path, 'wb') as f:
    num_sequences = len(X_unlabeled1_normalized)
    chunk_size = 100
    for i in range(0, num_sequences, chunk_size):
        chunk = X_unlabeled1_normalized[i:i + chunk_size]
        scaled_chunk = chunk * scaling_factor
        np.save(f, scaled_chunk)
        print(f"Processed and saved scaled chunk {i // chunk_size + 1} of {int(np.ceil(num_sequences / chunk_size))}")

# Check data ranges after scaling
def check_data_ranges(dataset, name):
    min_val = np.min(dataset)
    max_val = np.max(dataset)
    print(f"{name} - Min Value: {min_val}, Max Value: {max_val}")

check_data_ranges(np.load(scaled_output_path, mmap_mode='r'), "X_unlabeled1_scaled")
check_data_ranges(X_unlabeled2_normalized, "X_unlabeled2_normalized")
check_data_ranges(X_mediapipe_normalized, "X_mediapipe_normalized")

print(f"Scaled dataset saved to {scaled_output_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# Load the normalized datasets
X_unlabeled1_scaled = np.load('NEW_DATASET/scaled_interhand2.6m.npy', mmap_mode='r')
X_unlabeled2_normalized = np.load('NEW_DATASET/hsosunlabeled.npy', mmap_mode='r')
X_mediapipe_normalized = np.load('NEW_DATASET/dataset_20240709_X_3096.npy', mmap_mode='r')

# Print shapes of datasets
print(f"X_unlabeled1_scaled shape: {X_unlabeled1_scaled.shape}")
print(f"X_unlabeled2_normalized shape: {X_unlabeled2_normalized.shape}")
print(f"X_mediapipe_normalized shape: {X_mediapipe_normalized.shape}")

# Ensure all datasets are 3D (samples, 30, 126)
assert X_unlabeled1_scaled.ndim == 3 and X_unlabeled1_scaled.shape[2] == 126, "X_unlabeled1_scaled has incorrect shape"
assert X_unlabeled2_normalized.ndim == 3 and X_unlabeled2_normalized.shape[2] == 126, "X_unlabeled2_normalized has incorrect shape"
assert X_mediapipe_normalized.ndim == 3 and X_mediapipe_normalized.shape[2] == 126, "X_mediapipe_normalized has incorrect shape"

# Helper function to calculate descriptive statistics
def calculate_statistics(dataset, name):
    mean_val = np.mean(dataset)
    std_val = np.std(dataset)
    variance = np.var(dataset)
    print(f"{name} - Mean: {mean_val:.4f}, Std Dev: {std_val:.4f}, Variance: {variance:.4f}")
    return mean_val, std_val, variance

# Calculate statistics
print("\nDescriptive Statistics:")
stats_unlabeled1 = calculate_statistics(X_unlabeled1_scaled, "X_unlabeled1_scaled")
stats_unlabeled2 = calculate_statistics(X_unlabeled2_normalized, "X_unlabeled2_normalized")
stats_mediapipe = calculate_statistics(X_mediapipe_normalized, "X_mediapipe_normalized")

# Helper function to plot histograms
def plot_histograms(datasets, labels, bins=100):
    plt.figure(figsize=(12, 6))
    for dataset, label in zip(datasets, labels):
        plt.hist(dataset.flatten(), bins=bins, alpha=0.6, label=label, density=True)
    plt.title("Dataset Value Distributions")
    plt.xlabel("Value")
    plt.ylabel("Density")
    plt.legend()
    plt.show()

# Plot histograms
print("\nPlotting Histograms:")
plot_histograms(
    [X_unlabeled1_scaled, X_unlabeled2_normalized, X_mediapipe_normalized],
    ["X_unlabeled1_scaled", "X_unlabeled2_normalized", "X_mediapipe_normalized"]
)

# Perform Kolmogorov-Smirnov tests
def perform_ks_test(dataset1, dataset2, label1, label2):
    ks_stat, p_value = ks_2samp(dataset1.flatten(), dataset2.flatten())
    print(f"KS Test between {label1} and {label2}: KS Stat = {ks_stat:.4f}, P-value = {p_value:.4e}")

print("\nKolmogorov-Smirnov Tests:")
perform_ks_test(X_unlabeled1_scaled, X_unlabeled2_normalized, "X_unlabeled1_scaled", "X_unlabeled2_normalized")
perform_ks_test(X_unlabeled1_scaled, X_mediapipe_normalized, "X_unlabeled1_scaled", "X_mediapipe_normalized")
perform_ks_test(X_unlabeled2_normalized, X_mediapipe_normalized, "X_unlabeled2_normalized", "X_mediapipe_normalized")

# Visualize random samples
def visualize_random_samples(datasets, labels, num_samples=5):
    """
    Visualize random samples from datasets.

    Args:
    - datasets: List of datasets to visualize.
    - labels: Corresponding labels for the datasets.
    - num_samples: Number of samples to visualize.
    """
    fig, axes = plt.subplots(num_samples, len(datasets), figsize=(15, num_samples * 3))
    for i in range(num_samples):
        for j, (dataset, label) in enumerate(zip(datasets, labels)):
            # Select a random sequence and its first frame
            sequence_idx = np.random.randint(dataset.shape[0])
            frame = dataset[sequence_idx, 0].reshape(21, 3)  # Reshape the first frame (126 -> 21, 3)

            # Scatter plot for landmarks
            axes[i, j].scatter(frame[:, 0], frame[:, 1], c=frame[:, 2], cmap='viridis')
            axes[i, j].set_title(f"{label} - Sample {sequence_idx + 1}")
            axes[i, j].set_xlim(0, 1)
            axes[i, j].set_ylim(0, 1)
            axes[i, j].invert_yaxis()  # Match MediaPipe's visualization style
    plt.tight_layout()
    plt.show()

print("\nVisualizing Random Samples:")
visualize_random_samples(
    [X_unlabeled1_scaled, X_unlabeled2_normalized, X_mediapipe_normalized],
    ["X_unlabeled1_scaled", "X_unlabeled2_normalized", "X_mediapipe_normalized"]
)


# Pretrain with a giant dataset and save weights only the encoders. Next frame prediction with 15 frmaes

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

# Enable memory growth for GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Memory growth enabled for GPU devices.")
    except RuntimeError as e:
        print(f"Failed to set memory growth: {e}")
else:
    print("No GPU devices available.")

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow')

# Load the unlabeled datasets
X_unlabeled1 = np.load('NEW_DATASET/scaled_interhand2.6m.npy')  # (num_samples1, 30, 126)
X_unlabeled2 = np.load('NEW_DATASET/hsosunlabeled.npy')         # (num_samples2, 30, 126)

# Concatenate the arrays
X_unlabeled = np.concatenate((X_unlabeled1, X_unlabeled2), axis=0)
print("Shape of X_unlabeled:", X_unlabeled.shape)

# Choose which hand to pretrain on (0 for left/hand1, 1 for right/hand2)
hand_index = 0
if hand_index == 0:
    X_hand = X_unlabeled[:, :, :63]  # (num_samples, 30, 63)
    hand_name = "hand1"
else:
    X_hand = X_unlabeled[:, :, 63:]  # (num_samples, 30, 63)
    hand_name = "hand2"

# Split into inputs and outputs for next-frame prediction:
# First 15 frames as input, next 15 frames as output
X_input = X_hand[:, :15, :]   # (num_samples, 15, 63)
y_output = X_hand[:, 15:, :]  # (num_samples, 15, 63)

# Prepare decoder inputs by shifting y_output (not used directly here, but kept for reference)
decoder_inputs = np.zeros_like(y_output)
decoder_inputs[:, 1:, :] = y_output[:, :-1, :]  # Shifted right by one time step

print("Shape of X_input (encoder input):", X_input.shape)       # (num_samples, 15, 63)
print("Shape of decoder_inputs:", decoder_inputs.shape)         # (num_samples, 15, 63)
print("Shape of y_output (target):", y_output.shape)            # (num_samples, 15, 63)

# Transformer Encoder Definition
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1, name_prefix=""):
    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln1")(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha"
    )(x, x)
    x = layers.Dropout(dropout, name=f"{name_prefix}_dropout")(x)
    x = layers.Add(name=f"{name_prefix}_add")([x, inputs])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln2")(x)
    x_ff = layers.Dense(ff_dim, activation='gelu', name=f"{name_prefix}_dense1")(x)
    x_ff = layers.Dropout(dropout, name=f"{name_prefix}_dropout_ff")(x_ff)
    x_ff = layers.Dense(inputs.shape[-1], name=f"{name_prefix}_dense2")(x_ff)
    x = layers.Add(name=f"{name_prefix}_add_ff")([x_ff, x])
    return x

def build_pretraining_model(input_shape, head_size, num_heads, ff_dim, num_encoder_blocks, dropout=0.1):
    # input_shape here is (15, 63), meaning we operate on 15 timesteps and 63 features per hand
    left_input = tf.keras.Input(shape=input_shape, name="left_hand_input")
    right_input = tf.keras.Input(shape=input_shape, name="right_hand_input")

    # Left hand encoder
    x_left = left_input
    for i in range(num_encoder_blocks):
        x_left = transformer_encoder(
            x_left, head_size, num_heads, ff_dim, dropout, name_prefix=f"left_enc_{i}"
        )
    left_encoder_output = x_left  # (batch_size, 15, 63)

    # Right hand encoder
    x_right = right_input
    for i in range(num_encoder_blocks):
        x_right = transformer_encoder(
            x_right, head_size, num_heads, ff_dim, dropout, name_prefix=f"right_enc_{i}"
        )
    right_encoder_output = x_right  # (batch_size, 15, 63)

    # Concatenate outputs: (batch_size, 15, 63) + (batch_size, 15, 63) = (batch_size, 15, 126)
    concatenated_output = layers.Concatenate(name="concat_encoders")([left_encoder_output, right_encoder_output])

    # Output layer predicts the next frames for both hands (126 features total) over 15 timesteps
    output = layers.TimeDistributed(
        layers.Dense(126, activation='linear'), name="output"
    )(concatenated_output)

    model = tf.keras.Model(inputs=[left_input, right_input], outputs=output, name="pretraining_model")
    return model

# Define Hyperparameters
pretrain_hyperparameters = {
    'head_size': 64,
    'num_heads': 4,
    'ff_dim': 64,
    'num_encoder_blocks': 4,
    'dropout': 0.1,
    'learning_rate': 1e-3,
    'batch_size': 4096,
    'epochs': 200
}

# Prepare Model
# input_shape is (15,63) as per the data slicing done above
input_shape = (15, 63)
pretraining_model = build_pretraining_model(
    input_shape=input_shape,
    head_size=pretrain_hyperparameters['head_size'],
    num_heads=pretrain_hyperparameters['num_heads'],
    ff_dim=pretrain_hyperparameters['ff_dim'],
    num_encoder_blocks=pretrain_hyperparameters['num_encoder_blocks'],
    dropout=pretrain_hyperparameters['dropout']
)

# Freeze non-encoder layers if you ONLY want to train the encoder blocks
# In this model, only the output layer is non-encoder. Let's freeze it if desired:
for layer in pretraining_model.layers:
    # If you consider the output layer as non-encoder, freeze it:
    if layer.name.startswith("output"):
        layer.trainable = False
    # All layers with "enc_" prefix remain trainable (these are the encoder blocks)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0)
pretraining_model.compile(optimizer=optimizer, loss='mse')

# Learning rate schedule
def lr_schedule(epoch):
    warmup_epochs = 20
    total_epochs = pretrain_hyperparameters['epochs']
    initial_lr = pretrain_hyperparameters['learning_rate']
    min_lr = 1e-8

    if epoch < warmup_epochs:
        # Linear warm-up
        lr = initial_lr * (epoch + 1) / warmup_epochs
    else:
        # Linear decay
        decay_epochs = total_epochs - warmup_epochs
        lr = initial_lr - (initial_lr - min_lr) * (epoch - warmup_epochs + 1) / decay_epochs
        lr = max(lr, min_lr)
    print(f"Epoch {epoch+1}: Learning rate is {lr:.8f}")
    return lr

from tensorflow.keras.callbacks import LearningRateScheduler
callbacks = [LearningRateScheduler(lr_schedule, verbose=1)]

# Match target dimension with output dimension (126 features)
# We currently have y_output with shape (num_samples, 15, 63) for one hand.
# The model output is (num_samples, 15, 126) for both hands. Let's replicate:
y_output_full = np.concatenate([y_output, y_output], axis=-1)  # (num_samples, 15, 126)

# Since we have only one hand data in X_input, let's feed the same X_input to both left and right.
pretraining_model.fit(
    [X_input, X_input],  # Two inputs
    y_output_full,
    epochs=pretrain_hyperparameters['epochs'],
    batch_size=pretrain_hyperparameters['batch_size'],
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

pretraining_model.save_weights('encoder_pretrained_weights.h5')
print("Pretrained encoder weights saved.")


# Train the BiLSTM network with existing weights and labeled data

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

# Enable memory growth for GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU devices.")
    except RuntimeError as e:
        print(f"Failed to set memory growth: {e}")
else:
    print("No GPU devices available.")

# Data augmentation functions
def add_noise(data, noise_factor=0.0185):
    noise = np.random.randn(*data.shape) * noise_factor
    return np.clip(data + noise, 0, 1)

def scale(data, scale_factor=0.1605):
    scale = 1 + np.random.uniform(-scale_factor, scale_factor)
    return np.clip(data * scale, 0, 1)

def generate_random_curves(data, sigma=0.078, knot=2):
    from scipy.interpolate import CubicSpline
    xx = np.linspace(0, data.shape[0] - 1, knot + 2)
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, data.shape[1]))
    x_range = np.arange(data.shape[0])
    augmented_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        cs = CubicSpline(xx, yy[:, i])
        augmented_data[:, i] = data[:, i] * cs(x_range)
    return np.clip(augmented_data, 0, 1)

def augment_sample(sample):
    sample = add_noise(sample)
    sample = scale(sample)
    sample = generate_random_curves(sample)
    return sample

def augment_dataset(X, y, target_size=10000):
    num_original = len(X)
    num_augmented = target_size - num_original

    indices = np.random.randint(0, num_original, size=num_augmented)

    def process_sample(idx):
        augmented_x = augment_sample(X[idx])
        return augmented_x, y[idx]

    from joblib import Parallel, delayed
    augmented_samples = Parallel(n_jobs=-1)(
        delayed(process_sample)(idx) for idx in indices
    )

    augmented_X = np.array([sample[0] for sample in augmented_samples])
    augmented_y = np.array([sample[1] for sample in augmented_samples])
    return np.vstack((X, augmented_X)), np.vstack((y, augmented_y))

# Function to split hand data
def split_hand_data(X):
    hand1_data = X[:, :, :63]
    hand2_data = X[:, :, 63:]
    return hand1_data, hand2_data

# Function to load or create datasets for each subject
def load_or_create_subject_datasets(X, y, subject_number, split_indices, data_folder):
    # Determine file paths
    X_original_path = os.path.join(data_folder, f'X_subject{subject_number}_original.npy')
    y_original_path = os.path.join(data_folder, f'y_subject{subject_number}_original.npy')
    X_aug_path = os.path.join(data_folder, f'X_subject{subject_number}_aug.npy')
    y_aug_path = os.path.join(data_folder, f'y_subject{subject_number}_aug.npy')

    # Check if files exist
    files_exist = all(os.path.exists(p) for p in [X_original_path, y_original_path, X_aug_path, y_aug_path])

    if files_exist:
        print(f"Loading datasets for Subject {subject_number} from disk.")
        # Load datasets
        X_subject_original = np.load(X_original_path)
        y_subject_original = np.load(y_original_path)
        X_subject_aug = np.load(X_aug_path)
        y_subject_aug = np.load(y_aug_path)
    else:
        print(f"Creating and saving datasets for Subject {subject_number}.")
        # Split data for the subject
        start_idx, end_idx = split_indices
        X_subject_original = X[start_idx:end_idx]
        y_subject_original = y[start_idx:end_idx]
        # Save original data
        np.save(X_original_path, X_subject_original)
        np.save(y_original_path, y_subject_original)
        # Augment data
        X_subject_aug, y_subject_aug = augment_dataset(X_subject_original, y_subject_original, 10000)
        # Save augmented data
        np.save(X_aug_path, X_subject_aug)
        np.save(y_aug_path, y_subject_aug)

    return (X_subject_original, y_subject_original), (X_subject_aug, y_subject_aug)

input_shape_hand = (30, 63)  # Use full sequence length of 30

def prepare_datasets(train_subjects, test_subject):
    X_train_full = np.vstack([subject_datasets[train]["aug"][0] for train in train_subjects])
    y_train = np.vstack([subject_datasets[train]["aug"][1] for train in train_subjects])
    X_test_full, y_test = subject_datasets[test_subject]["original"]
    X_val_full, X_test_full, y_val, y_test = train_test_split(X_test_full, y_test, test_size=0.5, random_state=42)

    # Use full sequence length of 30
    X_train_full = X_train_full[:, :, :]
    X_val_full = X_val_full[:, :, :]
    X_test_full = X_test_full[:, :, :]

    # Split hand data for both hands
    X_train_hand1, X_train_hand2 = split_hand_data(X_train_full)
    X_val_hand1, X_val_hand2 = split_hand_data(X_val_full)
    X_test_hand1, X_test_hand2 = split_hand_data(X_test_full)

    return (X_train_hand1, X_train_hand2, y_train,
            X_val_hand1, X_val_hand2, y_val,
            X_test_hand1, X_test_hand2, y_test)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1, name_prefix=""):
    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln1")(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha"
    )(x, x)
    x = layers.Dropout(dropout, name=f"{name_prefix}_dropout")(x)
    x = layers.Add(name=f"{name_prefix}_add")([x, inputs])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln2")(x)
    x_ff = layers.Dense(ff_dim, activation='gelu', name=f"{name_prefix}_dense1")(x)
    x_ff = layers.Dropout(dropout, name=f"{name_prefix}_dropout_ff")(x_ff)
    x_ff = layers.Dense(inputs.shape[-1], name=f"{name_prefix}_dense2")(x_ff)
    x = layers.Add(name=f"{name_prefix}_add_ff")([x_ff, x])
    return x

def build_classification_model(input_shape_hand, head_size, num_heads, ff_dim, num_encoder_blocks, mlp_units, dropout=0.1):
    # Two separate inputs: left hand (hand1_input) and right hand (hand2_input)
    # Each hand has shape (30,63)
    hand1_input = tf.keras.Input(shape=input_shape_hand, name="hand1_input")
    hand2_input = tf.keras.Input(shape=input_shape_hand, name="hand2_input")

    # Pass each hand through the specified number of encoder blocks
    hand1_encoded = hand1_input
    hand2_encoded = hand2_input
    for i in range(num_encoder_blocks):
        hand1_encoded = transformer_encoder(
            hand1_encoded, head_size, num_heads, ff_dim, dropout, name_prefix=f"hand1_enc_{i}"
        )
        hand2_encoded = transformer_encoder(
            hand2_encoded, head_size, num_heads, ff_dim, dropout, name_prefix=f"hand2_enc_{i}"
        )

    # Concatenate encoder outputs along the feature dimension
    concatenated_output = layers.Concatenate(name="encoder_concat")([hand1_encoded, hand2_encoded])
    # concatenated_output shape: (batch_size, 30, 126)

    # Add a BiLSTM layer after the concatenation
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=False), name="bilstm")(concatenated_output)

    # Add two Dense layers after BiLSTM
    for idx, units in enumerate(mlp_units):
        x = layers.Dense(units, activation='gelu', name=f"mlp_dense_{idx}")(x)
        x = layers.Dropout(dropout, name=f"mlp_dropout_{idx}")(x)

    # Output layer for classification (9 classes)
    output = layers.Dense(9, activation='softmax', name="classification_output")(x)

    model = tf.keras.Model(inputs=[hand1_input, hand2_input], outputs=output, name="classification_model")
    return model

# Load the labeled dataset
X = np.load('NEW_DATASET/dataset_20240709_X_3096.npy')
y = np.load('NEW_DATASET/dataset_20240709_y_3096.npy')

# Create data folder if it doesn't exist
data_folder = 'processed_datasets'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Split indices for the subjects
split_60 = int(0.60 * len(X))
split_80 = int(0.80 * len(X))

# Load or create datasets for subjects
(X_subject1, y_subject1), (X_subject1_aug, y_subject1_aug) = load_or_create_subject_datasets(
    X, y, subject_number=1, split_indices=(0, split_60), data_folder=data_folder)

(X_subject2, y_subject2), (X_subject2_aug, y_subject2_aug) = load_or_create_subject_datasets(
    X, y, subject_number=2, split_indices=(split_60, split_80), data_folder=data_folder)

(X_subject3, y_subject3), (X_subject3_aug, y_subject3_aug) = load_or_create_subject_datasets(
    X, y, subject_number=3, split_indices=(split_80, len(X)), data_folder=data_folder)

# Now, store datasets in a dictionary
subject_datasets = {
    1: {"aug": (X_subject1_aug, y_subject1_aug), "original": (X_subject1, y_subject1)},
    2: {"aug": (X_subject2_aug, y_subject2_aug), "original": (X_subject2, y_subject2)},
    3: {"aug": (X_subject3_aug, y_subject3_aug), "original": (X_subject3, y_subject3)}
}

# Define hyperparameters for fine-tuning
fine_tune_hyperparameters = {
    'head_size': 64,
    'num_heads': 4,
    'ff_dim': 64,
    'num_encoder_blocks': 4,  # same as pretraining
    'mlp_units': [64, 32],    # after BiLSTM
    'dropout': 0.1,
    'learning_rate': 5e-4,
    'batch_size': 4096,
    'epochs': 200
}

# Prepare the combinations
combinations = [
    {'train_subjects': [1, 2], 'test_subject': 3},
    {'train_subjects': [1, 3], 'test_subject': 2},
    {'train_subjects': [2, 3], 'test_subject': 1}
]

# Initialize lists to store the metrics
test_losses = []
test_accuracies = []
f1_scores = []

for idx, combo in enumerate(combinations):
    print(f"\nRunning combination {idx+1}: Train on subjects {combo['train_subjects']}, Test on subject {combo['test_subject']}")

    train_subjects = combo['train_subjects']
    test_subject = combo['test_subject']

    # Prepare datasets
    X_train_hand1, X_train_hand2, y_train, \
    X_val_hand1, X_val_hand2, y_val, \
    X_test_hand1, X_test_hand2, y_test = prepare_datasets(train_subjects, test_subject)

    # Build the classification model
    classification_model = build_classification_model(
        input_shape_hand=input_shape_hand,
        head_size=fine_tune_hyperparameters['head_size'],
        num_heads=fine_tune_hyperparameters['num_heads'],
        ff_dim=fine_tune_hyperparameters['ff_dim'],
        num_encoder_blocks=fine_tune_hyperparameters['num_encoder_blocks'],
        mlp_units=fine_tune_hyperparameters['mlp_units'],
        dropout=fine_tune_hyperparameters['dropout']
    )

    # Load pretrained encoder weights
    classification_model.load_weights('encoder_pretrained_weights.h5', by_name=True)
    print("Pretrained encoder weights loaded.")

    # Freeze encoder layers and allow others to train
    """
    for layer in classification_model.layers:
        # If the layer name starts with "hand1_enc_" or "hand2_enc_", freeze it
        if layer.name.startswith("hand1_enc_") or layer.name.startswith("hand2_enc_"):
            layer.trainable = False
        else:
            layer.trainable = True
    """

    # Initialize the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0)

    # Compile the model
    classification_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    # Define the learning rate schedule function
    def lr_schedule(epoch):
        warmup_epochs = 20
        total_epochs = fine_tune_hyperparameters['epochs']
        initial_lr = fine_tune_hyperparameters['learning_rate']
        min_lr = 1e-8

        if epoch < warmup_epochs:
            # Linear warm-up
            lr = initial_lr * (epoch + 1) / warmup_epochs
        else:
            # Linear decay
            decay_epochs = total_epochs - warmup_epochs
            lr = initial_lr - (initial_lr - min_lr) * (epoch - warmup_epochs + 1) / decay_epochs
            lr = max(lr, min_lr)
        return lr

    from tensorflow.keras.callbacks import LearningRateScheduler

    # Create the LearningRateScheduler callback
    callbacks = [LearningRateScheduler(lr_schedule, verbose=0)]

    # Create datasets
    train_dataset = tf.data.Dataset.from_tensor_slices(
        ((X_train_hand1, X_train_hand2), y_train)
    ).shuffle(10000).batch(fine_tune_hyperparameters['batch_size']).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices(
        ((X_val_hand1, X_val_hand2), y_val)
    ).batch(fine_tune_hyperparameters['batch_size']).prefetch(tf.data.AUTOTUNE)

    # Fine-tune the model
    history = classification_model.fit(
        train_dataset,
        epochs=fine_tune_hyperparameters['epochs'],
        validation_data=val_dataset,
        callbacks=callbacks,
        verbose=2
    )

    # Evaluate on test data
    test_dataset = tf.data.Dataset.from_tensor_slices(
        ((X_test_hand1, X_test_hand2), y_test)
    ).batch(fine_tune_hyperparameters['batch_size']).prefetch(tf.data.AUTOTUNE)

    # Get predictions
    y_pred_probs = classification_model.predict(test_dataset)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Get true labels
    y_true = np.argmax(y_test, axis=1)

    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Evaluate the model to get loss and accuracy
    test_loss, test_accuracy = classification_model.evaluate(test_dataset, verbose=0)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, F1 Score: {f1:.4f}')

    # Collect the metrics
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    f1_scores.append(f1)

# Calculate and print the averages
avg_test_loss = np.mean(test_losses)
avg_test_accuracy = np.mean(test_accuracies)
avg_f1_score = np.mean(f1_scores)

print(f'\nAverage Test Loss: {avg_test_loss:.4f}')
print(f'Average Test Accuracy: {avg_test_accuracy:.4f}')
print(f'Average F1 Score: {avg_f1_score:.4f}')


# Pretrain with two bilstms and feed to decoder

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

# Enable memory growth for GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Memory growth enabled for GPU devices.")
    except RuntimeError as e:
        print(f"Failed to set memory growth: {e}")
else:
    print("No GPU devices available.")

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow')

# Load the unlabeled datasets
X_unlabeled1 = np.load('NEW_DATASET/scaled_interhand2.6m.npy')  # (num_samples1, 30, 126)
X_unlabeled2 = np.load('NEW_DATASET/hsosunlabeled.npy')         # (num_samples2, 30, 126)

# Concatenate the arrays
X_unlabeled = np.concatenate((X_unlabeled1, X_unlabeled2), axis=0)
print("Shape of X_unlabeled:", X_unlabeled.shape)

# Choose which hand to pretrain on (0 for left/hand1, 1 for right/hand2)
hand_index = 0
if hand_index == 0:
    X_hand = X_unlabeled[:, :, :63]  # (num_samples, 30, 63)
    hand_name = "hand1"
else:
    X_hand = X_unlabeled[:, :, 63:]  # (num_samples, 30, 63)
    hand_name = "hand2"

# Split into inputs and outputs for next-frame prediction
X_input = X_hand[:, :15, :]   # (num_samples, 15, 63)
y_output = X_hand[:, 15:, :]  # (num_samples, 15, 63)

# Prepare decoder inputs by shifting y_output (just for reference)
decoder_inputs = np.zeros_like(y_output)
decoder_inputs[:, 1:, :] = y_output[:, :-1, :]

print("Shape of X_input (encoder input):", X_input.shape)
print("Shape of decoder_inputs:", decoder_inputs.shape)
print("Shape of y_output (target):", y_output.shape)

def transformer_decoder_block(inputs, encoder_output, head_size, num_heads, ff_dim, dropout=0.1, name_prefix=""):
    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln1")(inputs)
    x1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha1"
    )(x, x)
    x1 = layers.Dropout(dropout, name=f"{name_prefix}_dropout1")(x1)
    x = layers.Add(name=f"{name_prefix}_add1")([x1, inputs])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln2")(x)
    x2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha2"
    )(x, encoder_output)
    x2 = layers.Dropout(dropout, name=f"{name_prefix}_dropout2")(x2)
    x = layers.Add(name=f"{name_prefix}_add2")([x2, x])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln3")(x)
    x_ff = layers.Dense(ff_dim, activation='gelu', name=f"{name_prefix}_dense1_ff")(x)
    x_ff = layers.Dropout(dropout, name=f"{name_prefix}_dropout_ff")(x_ff)
    x_ff = layers.Dense(inputs.shape[-1], name=f"{name_prefix}_dense2_ff")(x_ff)
    x = layers.Add(name=f"{name_prefix}_add_ff")([x_ff, x])

    return x

def build_pretraining_model(input_shape, head_size, num_heads, ff_dim, dropout=0.1, num_decoders=1):
    left_input = tf.keras.Input(shape=input_shape, name="left_hand_input")
    right_input = tf.keras.Input(shape=input_shape, name="right_hand_input")

    # BiLSTMs
    left_bilstm_output = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True),
        name="left_bilstm"
    )(left_input)

    right_bilstm_output = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True),
        name="right_bilstm"
    )(right_input)

    decoder_input = left_bilstm_output
    encoder_output = right_bilstm_output

    x = decoder_input
    for i in range(num_decoders):
        x = transformer_decoder_block(
            x, encoder_output,
            head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout,
            name_prefix=f"decoder_block_{i}"
        )

    output = layers.TimeDistributed(
        layers.Dense(126, activation='linear'),
        name="output"
    )(x)

    model = tf.keras.Model(inputs=[left_input, right_input], outputs=output, name="pretraining_model")
    return model

# Define Hyperparameters
pretrain_hyperparameters = {
    'head_size': 64,
    'num_heads': 4,
    'ff_dim': 64,
    'dropout': 0.1,
    'learning_rate': 1e-3,
    'batch_size': 2048,
    'epochs': 200,
    'num_decoders': 1  # Adjust this as needed
}

input_shape = (15, 63)
pretraining_model = build_pretraining_model(
    input_shape=input_shape,
    head_size=pretrain_hyperparameters['head_size'],
    num_heads=pretrain_hyperparameters['num_heads'],
    ff_dim=pretrain_hyperparameters['ff_dim'],
    dropout=pretrain_hyperparameters['dropout'],
    num_decoders=pretrain_hyperparameters['num_decoders']
)

# Freeze all layers except the BiLSTM layers
for layer in pretraining_model.layers:
    if layer.name not in ["left_bilstm", "right_bilstm"]:
        layer.trainable = False
    else:
        layer.trainable = True

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0)
pretraining_model.compile(optimizer=optimizer, loss='mse')

def lr_schedule(epoch):
    warmup_epochs = 20
    total_epochs = pretrain_hyperparameters['epochs']
    initial_lr = pretrain_hyperparameters['learning_rate']
    min_lr = 1e-8

    if epoch < warmup_epochs:
        lr = initial_lr * (epoch + 1) / warmup_epochs
    else:
        decay_epochs = total_epochs - warmup_epochs
        lr = initial_lr - (initial_lr - min_lr) * (epoch - warmup_epochs + 1) / decay_epochs
        lr = max(lr, min_lr)
    print(f"Epoch {epoch+1}: Learning rate is {lr:.8f}")
    return lr

from tensorflow.keras.callbacks import LearningRateScheduler
callbacks = [LearningRateScheduler(lr_schedule, verbose=1)]

# Prepare target data
y_output_full = np.concatenate([y_output, y_output], axis=-1)

pretraining_model.fit(
    [X_input, X_input],
    y_output_full,
    epochs=pretrain_hyperparameters['epochs'],
    batch_size=pretrain_hyperparameters['batch_size'],
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

# Extract only the BiLSTM weights
left_bilstm_layer = pretraining_model.get_layer('left_bilstm')
right_bilstm_layer = pretraining_model.get_layer('right_bilstm')

left_weights = left_bilstm_layer.get_weights()
right_weights = right_bilstm_layer.get_weights()

# Save only the BiLSTM weights
np.savez('bilstm_pretrained_weights.npz',
         left_bilstm=left_weights,
         right_bilstm=right_weights)

print("BiLSTM pretrained weights saved independently of the decoder block.")


# Use pretrained weights of double bilstm to decoder

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

# Enable memory growth for GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU devices.")
    except RuntimeError as e:
        print(f"Failed to set memory growth: {e}")
else:
    print("No GPU devices available.")

def add_noise(data, noise_factor=0.0185):
    noise = np.random.randn(*data.shape) * noise_factor
    return np.clip(data + noise, 0, 1)

def scale(data, scale_factor=0.1605):
    scale = 1 + np.random.uniform(-scale_factor, scale_factor)
    return np.clip(data * scale, 0, 1)

def generate_random_curves(data, sigma=0.078, knot=2):
    from scipy.interpolate import CubicSpline
    xx = np.linspace(0, data.shape[0] - 1, knot + 2)
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, data.shape[1]))
    x_range = np.arange(data.shape[0])
    augmented_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        cs = CubicSpline(xx, yy[:, i])
        augmented_data[:, i] = data[:, i] * cs(x_range)
    return np.clip(augmented_data, 0, 1)

def augment_sample(sample):
    sample = add_noise(sample)
    sample = scale(sample)
    sample = generate_random_curves(sample)
    return sample

def augment_dataset(X, y, target_size=10000):
    num_original = len(X)
    num_augmented = target_size - num_original
    indices = np.random.randint(0, num_original, size=num_augmented)

    from joblib import Parallel, delayed
    augmented_samples = Parallel(n_jobs=-1)(
        delayed(lambda idx: (augment_sample(X[idx]), y[idx]))(idx) for idx in indices
    )

    augmented_X = np.array([s[0] for s in augmented_samples])
    augmented_y = np.array([s[1] for s in augmented_samples])
    return np.vstack((X, augmented_X)), np.vstack((y, augmented_y))

def split_hand_data(X):
    hand1_data = X[:, :, :63]
    hand2_data = X[:, :, 63:]
    return hand1_data, hand2_data

def load_or_create_subject_datasets(X, y, subject_number, split_indices, data_folder):
    X_original_path = os.path.join(data_folder, f'X_subject{subject_number}_original.npy')
    y_original_path = os.path.join(data_folder, f'y_subject{subject_number}_original.npy')
    X_aug_path = os.path.join(data_folder, f'X_subject{subject_number}_aug.npy')
    y_aug_path = os.path.join(data_folder, f'y_subject{subject_number}_aug.npy')

    files_exist = all(os.path.exists(p) for p in [X_original_path, y_original_path, X_aug_path, y_aug_path])

    if files_exist:
        print(f"Loading datasets for Subject {subject_number} from disk.")
        X_subject_original = np.load(X_original_path)
        y_subject_original = np.load(y_original_path)
        X_subject_aug = np.load(X_aug_path)
        y_subject_aug = np.load(y_aug_path)
    else:
        print(f"Creating and saving datasets for Subject {subject_number}.")
        start_idx, end_idx = split_indices
        X_subject_original = X[start_idx:end_idx]
        y_subject_original = y[start_idx:end_idx]
        np.save(X_original_path, X_subject_original)
        np.save(y_original_path, y_subject_original)
        X_subject_aug, y_subject_aug = augment_dataset(X_subject_original, y_subject_original, 10000)
        np.save(X_aug_path, X_subject_aug)
        np.save(y_aug_path, y_aug_path)

    return (X_subject_original, y_subject_original), (X_subject_aug, y_subject_aug)

input_shape_hand = (30, 63)

def prepare_datasets(train_subjects, test_subject):
    X_train_full = np.vstack([subject_datasets[train]["aug"][0] for train in train_subjects])
    y_train = np.vstack([subject_datasets[train]["aug"][1] for train in train_subjects])
    X_test_full, y_test = subject_datasets[test_subject]["original"]
    X_val_full, X_test_full, y_val, y_test = train_test_split(X_test_full, y_test, test_size=0.5, random_state=42)

    X_train_hand1, X_train_hand2 = split_hand_data(X_train_full)
    X_val_hand1, X_val_hand2 = split_hand_data(X_val_full)
    X_test_hand1, X_test_hand2 = split_hand_data(X_test_full)

    return (X_train_hand1, X_train_hand2, y_train,
            X_val_hand1, X_val_hand2, y_val,
            X_test_hand1, X_test_hand2, y_test)

def transformer_decoder_block(inputs, encoder_output, head_size, num_heads, ff_dim, dropout=0.1, name_prefix=""):
    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln1")(inputs)
    x1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha1"
    )(x, x)
    x1 = layers.Dropout(dropout, name=f"{name_prefix}_dropout1")(x1)
    x = layers.Add(name=f"{name_prefix}_add1")([x1, inputs])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln2")(x)
    x2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha2"
    )(x, encoder_output)
    x2 = layers.Dropout(dropout, name=f"{name_prefix}_dropout2")(x2)
    x = layers.Add(name=f"{name_prefix}_add2")([x2, x])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln3")(x)
    x_ff = layers.Dense(ff_dim, activation='gelu', name=f"{name_prefix}_dense1_ff")(x)
    x_ff = layers.Dropout(dropout, name=f"{name_prefix}_dropout_ff")(x_ff)
    x_ff = layers.Dense(inputs.shape[-1], name=f"{name_prefix}_dense2_ff")(x_ff)
    x = layers.Add(name=f"{name_prefix}_add_ff")([x_ff, x])

    return x

def build_pretrained_structure(input_shape, head_size, num_heads, ff_dim, dropout=0.1, num_decoders=1):
    left_input = tf.keras.Input(shape=input_shape, name="left_hand_input")
    right_input = tf.keras.Input(shape=input_shape, name="right_hand_input")

    left_bilstm_output = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True),
        name="left_bilstm"
    )(left_input)

    right_bilstm_output = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True),
        name="right_bilstm"
    )(right_input)

    decoder_input = left_bilstm_output
    encoder_output = right_bilstm_output

    x = decoder_input
    for i in range(num_decoders):
        x = transformer_decoder_block(
            x, encoder_output,
            head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout,
            name_prefix=f"decoder_block_{i}"
        )

    output = layers.TimeDistributed(
        layers.Dense(126, activation='linear'), name="output"
    )(x)

    model = tf.keras.Model(inputs=[left_input, right_input], outputs=output, name="pretrained_structure")
    return model

def build_classification_model_from_pretrained(pretrained_model, mlp_units=[128,64], dropout=0.1, num_decoders=1):
    # Always use "decoder_block_{num_decoders-1}"
    last_decoder_prefix = f"decoder_block_{num_decoders-1}"
    decoder_output_layer = pretrained_model.get_layer(last_decoder_prefix + "_add_ff")
    decoder_output = decoder_output_layer.output

    x = layers.Flatten(name="flatten")(decoder_output)
    for idx, units in enumerate(mlp_units):
        x = layers.Dense(units, activation='gelu', name=f"mlp_dense_{idx}")(x)
        x = layers.Dropout(dropout, name=f"mlp_dropout_{idx}")(x)

    output = layers.Dense(9, activation='softmax', name="classification_output")(x)
    classification_model = tf.keras.Model(inputs=pretrained_model.inputs, outputs=output, name="classification_model")
    return classification_model


X = np.load('NEW_DATASET/dataset_20240709_X_3096.npy')
y = np.load('NEW_DATASET/dataset_20240709_y_3096.npy')

data_folder = 'processed_datasets'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

split_60 = int(0.60 * len(X))
split_80 = int(0.80 * len(X))

(X_subject1, y_subject1), (X_subject1_aug, y_subject1_aug) = load_or_create_subject_datasets(
    X, y, subject_number=1, split_indices=(0, split_60), data_folder=data_folder)

(X_subject2, y_subject2), (X_subject2_aug, y_subject2_aug) = load_or_create_subject_datasets(
    X, y, subject_number=2, split_indices=(split_60, split_80), data_folder=data_folder)

(X_subject3, y_subject3), (X_subject3_aug, y_subject3_aug) = load_or_create_subject_datasets(
    X, y, subject_number=3, split_indices=(split_80, len(X)), data_folder=data_folder)

subject_datasets = {
    1: {"aug": (X_subject1_aug, y_subject1_aug), "original": (X_subject1, y_subject1)},
    2: {"aug": (X_subject2_aug, y_subject2_aug), "original": (X_subject2, y_subject2)},
    3: {"aug": (X_subject3_aug, y_subject3_aug), "original": (X_subject3, y_subject3)}
}

# Add num_decoders as a hyperparameter
head_sizes = [64]
num_heads_list = [4]
ff_dims = [128]
dropouts = [0.1, 0.2]
learning_rates = np.linspace(2e-4, 1e-3, 3) 
batch_sizes = [2048]
epochs = 200
num_decoders_list = [1, 2, 3]  # Example values for number of decoders

combinations_subjects = [
    {'train_subjects': [1, 2], 'test_subject': 3},
    {'train_subjects': [1, 3], 'test_subject': 2},
    {'train_subjects': [2, 3], 'test_subject': 1}
]

for head_size in head_sizes:
    for num_heads in num_heads_list:
        for ff_dim in ff_dims:
            for dropout in dropouts:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        for num_decoders in num_decoders_list:
                            print("\n======================================")
                            print(f"Testing hyperparameters: head_size={head_size}, num_heads={num_heads}, ff_dim={ff_dim}, dropout={dropout}, lr={learning_rate}, batch_size={batch_size}, epochs={epochs}, num_decoders={num_decoders}")
                            print("======================================\n")

                            test_losses = []
                            test_accuracies = []
                            f1_scores = []

                            for idx, combo in enumerate(combinations_subjects):
                                print(f"\nRunning combination {idx+1}: Train on subjects {combo['train_subjects']}, Test on subject {combo['test_subject']}")

                                train_subjects = combo['train_subjects']
                                test_subject = combo['test_subject']

                                X_train_hand1, X_train_hand2, y_train, \
                                X_val_hand1, X_val_hand2, y_val, \
                                X_test_hand1, X_test_hand2, y_test = prepare_datasets(train_subjects, test_subject)

                                pretrained_structure = build_pretrained_structure(
                                    input_shape_hand,
                                    head_size=head_size,
                                    num_heads=num_heads,
                                    ff_dim=ff_dim,
                                    dropout=dropout,
                                    num_decoders=num_decoders
                                )

                                # Load only BiLSTM weights
                                bilstm_weights = np.load('bilstm_pretrained_weights.npz', allow_pickle=True)
                                left_weights = bilstm_weights['left_bilstm']
                                right_weights = bilstm_weights['right_bilstm']

                                pretrained_structure.get_layer('left_bilstm').set_weights(left_weights)
                                pretrained_structure.get_layer('right_bilstm').set_weights(right_weights)

                                print("BiLSTM pretrained weights loaded.")

                                classification_model = build_classification_model_from_pretrained(
                                    pretrained_structure,
                                    mlp_units=[128,64],
                                    dropout=dropout,
                                    num_decoders=num_decoders
                                )

                                # BiLSTM weights are already set in pretrained_structure and classification_model shares them
                                # Freeze BiLSTM layers so they are NOT trained
                                for layer in classification_model.layers:
                                    if 'bilstm' in layer.name:
                                        layer.trainable = False
                                    else:
                                        layer.trainable = True

                                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0)
                                classification_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

                                def lr_schedule(epoch):
                                    warmup_epochs = 20
                                    total_epochs = epochs
                                    initial_lr = learning_rate
                                    min_lr = 1e-8

                                    if epoch < warmup_epochs:
                                        lr = initial_lr * (epoch + 1) / warmup_epochs
                                    else:
                                        decay_epochs = total_epochs - warmup_epochs
                                        lr = initial_lr - (initial_lr - min_lr) * (epoch - warmup_epochs + 1) / decay_epochs
                                        lr = max(lr, min_lr)
                                    return lr

                                from tensorflow.keras.callbacks import LearningRateScheduler
                                callbacks = [LearningRateScheduler(lr_schedule, verbose=0)]

                                train_dataset = tf.data.Dataset.from_tensor_slices(
                                    ((X_train_hand1, X_train_hand2), y_train)
                                ).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                                val_dataset = tf.data.Dataset.from_tensor_slices(
                                    ((X_val_hand1, X_val_hand2), y_val)
                                ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                                history = classification_model.fit(
                                    train_dataset,
                                    epochs=epochs,
                                    validation_data=val_dataset,
                                    callbacks=callbacks,
                                    verbose=0
                                )

                                test_dataset = tf.data.Dataset.from_tensor_slices(
                                    ((X_test_hand1, X_test_hand2), y_test)
                                ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                                y_pred_probs = classification_model.predict(test_dataset)
                                y_pred = np.argmax(y_pred_probs, axis=1)
                                y_true = np.argmax(y_test, axis=1)

                                f1 = f1_score(y_true, y_pred, average='weighted')
                                test_loss, test_accuracy = classification_model.evaluate(test_dataset, verbose=0)
                                print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, F1 Score: {f1:.4f}')

                                test_losses.append(test_loss)
                                test_accuracies.append(test_accuracy)
                                f1_scores.append(f1)

                            # Average metrics over three subject combinations
                            avg_test_loss = np.mean(test_losses)
                            avg_test_accuracy = np.mean(test_accuracies)
                            avg_f1_score = np.mean(f1_scores)

                            print("\n*** AVERAGED RESULTS FOR THIS HYPERPARAMETER SET ***")
                            print(f'Average Test Loss: {avg_test_loss:.4f}')
                            print(f'Average Test Accuracy: {avg_test_accuracy:.4f}')
                            print(f'Average F1 Score: {avg_f1_score:.4f}')
                            print("****************************************************\n")


# Train from scratch with BiLSTM and deocder

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

# Enable memory growth for GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.set_visible_devices(physical_gpus[0], 'GPU')
        print("Memory growth enabled for GPU devices.")
    except RuntimeError as e:
        print(f"Failed to set memory growth: {e}")
else:
    print("No GPU devices available.")

def add_noise(data, noise_factor=0.0185):
    noise = np.random.randn(*data.shape) * noise_factor
    return np.clip(data + noise, 0, 1)

def scale(data, scale_factor=0.1605):
    scale = 1 + np.random.uniform(-scale_factor, scale_factor)
    return np.clip(data * scale, 0, 1)

def generate_random_curves(data, sigma=0.078, knot=2):
    from scipy.interpolate import CubicSpline
    xx = np.linspace(0, data.shape[0] - 1, knot + 2)
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, data.shape[1]))
    x_range = np.arange(data.shape[0])
    augmented_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        cs = CubicSpline(xx, yy[:, i])
        augmented_data[:, i] = data[:, i] * cs(x_range)
    return np.clip(augmented_data, 0, 1)

def augment_sample(sample):
    sample = add_noise(sample)
    sample = scale(sample)
    sample = generate_random_curves(sample)
    return sample

def augment_dataset(X, y, target_size=10000):
    num_original = len(X)
    num_augmented = target_size - num_original
    indices = np.random.randint(0, num_original, size=num_augmented)

    from joblib import Parallel, delayed
    augmented_samples = Parallel(n_jobs=-1)(
        delayed(lambda idx: (augment_sample(X[idx]), y[idx]))(idx) for idx in indices
    )

    augmented_X = np.array([s[0] for s in augmented_samples])
    augmented_y = np.array([s[1] for s in augmented_samples])
    return np.vstack((X, augmented_X)), np.vstack((y, augmented_y))

def split_hand_data(X):
    hand1_data = X[:, :, :63]
    hand2_data = X[:, :, 63:]
    return hand1_data, hand2_data

def load_or_create_subject_datasets(X, y, subject_number, split_indices, data_folder):
    X_original_path = os.path.join(data_folder, f'X_subject{subject_number}_original.npy')
    y_original_path = os.path.join(data_folder, f'y_subject{subject_number}_original.npy')
    X_aug_path = os.path.join(data_folder, f'X_subject{subject_number}_aug.npy')
    y_aug_path = os.path.join(data_folder, f'y_subject{subject_number}_aug.npy')

    files_exist = all(os.path.exists(p) for p in [X_original_path, y_original_path, X_aug_path, y_aug_path])

    if files_exist:
        print(f"Loading datasets for Subject {subject_number} from disk.")
        X_subject_original = np.load(X_original_path)
        y_subject_original = np.load(y_original_path)
        X_subject_aug = np.load(X_aug_path)
        y_subject_aug = np.load(y_aug_path)
    else:
        print(f"Creating and saving datasets for Subject {subject_number}.")
        start_idx, end_idx = split_indices
        X_subject_original = X[start_idx:end_idx]
        y_subject_original = y[start_idx:end_idx]
        np.save(X_original_path, X_subject_original)
        np.save(y_original_path, y_subject_original)
        X_subject_aug, y_subject_aug = augment_dataset(X_subject_original, y_subject_original, 10000)
        np.save(X_aug_path, X_subject_aug)
        np.save(y_aug_path, y_aug_path)

    return (X_subject_original, y_subject_original), (X_subject_aug, y_subject_aug)

input_shape_hand = (30, 63)

def prepare_datasets(train_subjects, test_subject):
    X_train_full = np.vstack([subject_datasets[train]["aug"][0] for train in train_subjects])
    y_train = np.vstack([subject_datasets[train]["aug"][1] for train in train_subjects])
    X_test_full, y_test = subject_datasets[test_subject]["original"]
    X_val_full, X_test_full, y_val, y_test = train_test_split(X_test_full, y_test, test_size=0.5, random_state=42)

    X_train_hand1, X_train_hand2 = split_hand_data(X_train_full)
    X_val_hand1, X_val_hand2 = split_hand_data(X_val_full)
    X_test_hand1, X_test_hand2 = split_hand_data(X_test_full)

    return (X_train_hand1, X_train_hand2, y_train,
            X_val_hand1, X_val_hand2, y_val,
            X_test_hand1, X_test_hand2, y_test)

def transformer_decoder_block(inputs, encoder_output, head_size, num_heads, ff_dim, dropout=0.1, name_prefix=""):
    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln1")(inputs)
    x1 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha1"
    )(x, x)
    x1 = layers.Dropout(dropout, name=f"{name_prefix}_dropout1")(x1)
    x = layers.Add(name=f"{name_prefix}_add1")([x1, inputs])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln2")(x)
    x2 = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout, name=f"{name_prefix}_mha2"
    )(x, encoder_output)
    x2 = layers.Dropout(dropout, name=f"{name_prefix}_dropout2")(x2)
    x = layers.Add(name=f"{name_prefix}_add2")([x2, x])

    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_ln3")(x)
    x_ff = layers.Dense(ff_dim, activation='gelu', name=f"{name_prefix}_dense1_ff")(x)
    x_ff = layers.Dropout(dropout, name=f"{name_prefix}_dropout_ff")(x_ff)
    x_ff = layers.Dense(inputs.shape[-1], name=f"{name_prefix}_dense2_ff")(x_ff)
    x = layers.Add(name=f"{name_prefix}_add_ff")([x_ff, x])

    return x

def build_pretrained_structure(input_shape, head_size, num_heads, ff_dim, dropout=0.1, num_decoders=1, bilstm_unit_size=256):
    left_input = tf.keras.Input(shape=input_shape, name="left_hand_input")
    right_input = tf.keras.Input(shape=input_shape, name="right_hand_input")

    left_bilstm_output = layers.Bidirectional(
        layers.LSTM(bilstm_unit_size, return_sequences=True),
        name="left_bilstm"
    )(left_input)

    right_bilstm_output = layers.Bidirectional(
        layers.LSTM(bilstm_unit_size, return_sequences=True),
        name="right_bilstm"
    )(right_input)

    decoder_input = left_bilstm_output
    encoder_output = right_bilstm_output

    x = decoder_input
    for i in range(num_decoders):
        x = transformer_decoder_block(
            x, encoder_output,
            head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout,
            name_prefix=f"decoder_block_{i}"
        )

    output = layers.TimeDistributed(
        layers.Dense(126, activation='linear'), name="output"
    )(x)

    model = tf.keras.Model(inputs=[left_input, right_input], outputs=output, name="pretrained_structure")
    return model

def build_classification_model_from_pretrained(pretrained_model, mlp_units=[128,64], dropout=0.1, num_decoders=1):
    # Always use "decoder_block_{num_decoders-1}"
    last_decoder_prefix = f"decoder_block_{num_decoders-1}"
    decoder_output_layer = pretrained_model.get_layer(last_decoder_prefix + "_add_ff")
    decoder_output = decoder_output_layer.output

    x = layers.Flatten(name="flatten")(decoder_output)
    for idx, units in enumerate(mlp_units):
        x = layers.Dense(units, activation='gelu', name=f"mlp_dense_{idx}")(x)
        x = layers.Dropout(dropout, name=f"mlp_dropout_{idx}")(x)

    output = layers.Dense(9, activation='softmax', name="classification_output")(x)
    classification_model = tf.keras.Model(inputs=pretrained_model.inputs, outputs=output, name="classification_model")
    return classification_model

X = np.load('NEW_DATASET/dataset_20240709_X_3096.npy')
y = np.load('NEW_DATASET/dataset_20240709_y_3096.npy')

data_folder = 'processed_datasets'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

split_60 = int(0.60 * len(X))
split_80 = int(0.80 * len(X))

(X_subject1, y_subject1), (X_subject1_aug, y_subject1_aug) = load_or_create_subject_datasets(
    X, y, subject_number=1, split_indices=(0, split_60), data_folder=data_folder)

(X_subject2, y_subject2), (X_subject2_aug, y_subject2_aug) = load_or_create_subject_datasets(
    X, y, subject_number=2, split_indices=(split_60, split_80), data_folder=data_folder)

(X_subject3, y_subject3), (X_subject3_aug, y_subject3_aug) = load_or_create_subject_datasets(
    X, y, subject_number=3, split_indices=(split_80, len(X)), data_folder=data_folder)

subject_datasets = {
    1: {"aug": (X_subject1_aug, y_subject1_aug), "original": (X_subject1, y_subject1)},
    2: {"aug": (X_subject2_aug, y_subject2_aug), "original": (X_subject2, y_subject2)},
    3: {"aug": (X_subject3_aug, y_subject3_aug), "original": (X_subject3, y_subject3)}
}

# Add bilstm_unit_size as a hyperparameter
bilstm_unit_sizes = [128, 256]  # Example sizes to tune
head_sizes = [64,128]
num_heads_list = [4]
ff_dims = [64]
dropouts = [0.2]
learning_rates = np.linspace(2e-4, 8e-4, 3) 
batch_sizes = [2048]
epochs = 200
num_decoders_list = [1]  # Example values for number of decoders

combinations_subjects = [
    {'train_subjects': [1, 2], 'test_subject': 3},
    {'train_subjects': [1, 3], 'test_subject': 2},
    {'train_subjects': [2, 3], 'test_subject': 1}
]

for bilstm_unit_size in bilstm_unit_sizes:
    for head_size in head_sizes:
        for num_heads in num_heads_list:
            for ff_dim in ff_dims:
                for dropout in dropouts:
                    for learning_rate in learning_rates:
                        for batch_size in batch_sizes:
                            for num_decoders in num_decoders_list:
                                print("\n======================================")
                                print(f"Testing hyperparameters: bilstm_unit_size={bilstm_unit_size}, head_size={head_size}, num_heads={num_heads}, ff_dim={ff_dim}, dropout={dropout}, lr={learning_rate}, batch_size={batch_size}, epochs={epochs}, num_decoders={num_decoders}")
                                print("======================================\n")

                                test_losses = []
                                test_accuracies = []
                                f1_scores = []

                                for idx, combo in enumerate(combinations_subjects):
                                    print(f"\nRunning combination {idx+1}: Train on subjects {combo['train_subjects']}, Test on subject {combo['test_subject']}")

                                    train_subjects = combo['train_subjects']
                                    test_subject = combo['test_subject']

                                    X_train_hand1, X_train_hand2, y_train, \
                                    X_val_hand1, X_val_hand2, y_val, \
                                    X_test_hand1, X_test_hand2, y_test = prepare_datasets(train_subjects, test_subject)

                                    pretrained_structure = build_pretrained_structure(
                                        input_shape_hand,
                                        head_size=head_size,
                                        num_heads=num_heads,
                                        ff_dim=ff_dim,
                                        dropout=dropout,
                                        num_decoders=num_decoders,
                                        bilstm_unit_size=bilstm_unit_size
                                    )

                                    # Train from scratch: no pretrained weights, no freezing
                                    classification_model = build_classification_model_from_pretrained(
                                        pretrained_structure,
                                        mlp_units=[128,64],
                                        dropout=dropout,
                                        num_decoders=num_decoders
                                    )

                                    for layer in classification_model.layers:
                                        layer.trainable = True

                                    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0)
                                    classification_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

                                    def lr_schedule(epoch):
                                        warmup_epochs = 20
                                        total_epochs = epochs
                                        initial_lr = learning_rate
                                        min_lr = 1e-8

                                        if epoch < warmup_epochs:
                                            lr = initial_lr * (epoch + 1) / warmup_epochs
                                        else:
                                            decay_epochs = total_epochs - warmup_epochs
                                            lr = initial_lr - (initial_lr - min_lr) * (epoch - warmup_epochs + 1) / decay_epochs
                                            lr = max(lr, min_lr)
                                        return lr

                                    from tensorflow.keras.callbacks import LearningRateScheduler
                                    callbacks = [LearningRateScheduler(lr_schedule, verbose=0)]

                                    train_dataset = tf.data.Dataset.from_tensor_slices(
                                        ((X_train_hand1, X_train_hand2), y_train)
                                    ).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                                    val_dataset = tf.data.Dataset.from_tensor_slices(
                                        ((X_val_hand1, X_val_hand2), y_val)
                                    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                                    history = classification_model.fit(
                                        train_dataset,
                                        epochs=epochs,
                                        validation_data=val_dataset,
                                        callbacks=callbacks,
                                        verbose=0
                                    )

                                    test_dataset = tf.data.Dataset.from_tensor_slices(
                                        ((X_test_hand1, X_test_hand2), y_test)
                                    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                                    y_pred_probs = classification_model.predict(test_dataset)
                                    y_pred = np.argmax(y_pred_probs, axis=1)
                                    y_true = np.argmax(y_test, axis=1)

                                    f1 = f1_score(y_true, y_pred, average='weighted')
                                    test_loss, test_accuracy = classification_model.evaluate(test_dataset, verbose=0)
                                    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, F1 Score: {f1:.4f}')

                                    test_losses.append(test_loss)
                                    test_accuracies.append(test_accuracy)
                                    f1_scores.append(f1)

                                # Average metrics over three subject combinations
                                avg_test_loss = np.mean(test_losses)
                                avg_test_accuracy = np.mean(test_accuracies)
                                avg_f1_score = np.mean(f1_scores)

                                print("\n*** AVERAGED RESULTS FOR THIS HYPERPARAMETER SET ***")
                                print(f'Average Test Loss: {avg_test_loss:.4f}')
                                print(f'Average Test Accuracy: {avg_test_accuracy:.4f}')
                                print(f'Average F1 Score: {avg_f1_score:.4f}')
                                print("****************************************************\n")


# Two BiLSTMs, one flatten

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

def add_noise(data, noise_factor=0.0185):
    noise = np.random.randn(*data.shape) * noise_factor
    return np.clip(data + noise, 0, 1)

def scale(data, scale_factor=0.1605):
    scale = 1 + np.random.uniform(-scale_factor, scale_factor)
    return np.clip(data * scale, 0, 1)

def generate_random_curves(data, sigma=0.078, knot=2):
    from scipy.interpolate import CubicSpline
    xx = np.linspace(0, data.shape[0] - 1, knot + 2)
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, data.shape[1]))
    x_range = np.arange(data.shape[0])
    augmented_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        cs = CubicSpline(xx, yy[:, i])
        augmented_data[:, i] = data[:, i] * cs(x_range)
    return np.clip(augmented_data, 0, 1)

def augment_sample(sample):
    sample = add_noise(sample)
    sample = scale(sample)
    sample = generate_random_curves(sample)
    return sample

def augment_dataset(X, y, target_size=10000):
    num_original = len(X)
    num_augmented = target_size - num_original
    indices = np.random.randint(0, num_original, size=num_augmented)

    from joblib import Parallel, delayed
    augmented_samples = Parallel(n_jobs=-1)(
        delayed(lambda idx: (augment_sample(X[idx]), y[idx]))(idx) for idx in indices
    )

    augmented_X = np.array([s[0] for s in augmented_samples])
    augmented_y = np.array([s[1] for s in augmented_samples])
    return np.vstack((X, augmented_X)), np.vstack((y, augmented_y))

def split_hand_data(X):
    hand1_data = X[:, :, :63]
    hand2_data = X[:, :, 63:]
    return hand1_data, hand2_data

def load_or_create_subject_datasets(X, y, subject_number, split_indices, data_folder):
    X_original_path = os.path.join(data_folder, f'X_subject{subject_number}_original.npy')
    y_original_path = os.path.join(data_folder, f'y_subject{subject_number}_original.npy')
    X_aug_path = os.path.join(data_folder, f'X_subject{subject_number}_aug.npy')
    y_aug_path = os.path.join(data_folder, f'y_subject{subject_number}_aug.npy')

    files_exist = all(os.path.exists(p) for p in [X_original_path, y_original_path, X_aug_path, y_aug_path])

    if files_exist:
        print(f"Loading datasets for Subject {subject_number} from disk.")
        X_subject_original = np.load(X_original_path)
        y_subject_original = np.load(y_original_path)
        X_subject_aug = np.load(X_aug_path)
        y_subject_aug = np.load(y_aug_path)
    else:
        print(f"Creating and saving datasets for Subject {subject_number}.")
        start_idx, end_idx = split_indices
        X_subject_original = X[start_idx:end_idx]
        y_subject_original = y[start_idx:end_idx]
        np.save(X_original_path, X_subject_original)
        np.save(y_original_path, y_subject_original)
        X_subject_aug, y_subject_aug = augment_dataset(X_subject_original, y_subject_original, 10000)
        np.save(X_aug_path, X_subject_aug)
        np.save(y_aug_path, y_aug_path)

    return (X_subject_original, y_subject_original), (X_subject_aug, y_subject_aug)

input_shape = (30, 63)

def prepare_datasets(train_subjects, test_subject):
    X_train_full = np.vstack([subject_datasets[train]["aug"][0] for train in train_subjects])
    y_train = np.vstack([subject_datasets[train]["aug"][1] for train in train_subjects])
    X_test_full, y_test = subject_datasets[test_subject]["original"]
    X_val_full, X_test_full, y_val, y_test = train_test_split(X_test_full, y_test, test_size=0.5, random_state=42)

    X_train_hand1, X_train_hand2 = split_hand_data(X_train_full)
    X_val_hand1, X_val_hand2 = split_hand_data(X_val_full)
    X_test_hand1, X_test_hand2 = split_hand_data(X_test_full)

    return (X_train_hand1, X_train_hand2, y_train,
            X_val_hand1, X_val_hand2, y_val,
            X_test_hand1, X_test_hand2, y_test)

def build_classification_model(bilstm_unit_size=128, mlp_units=[128,64], dropout=0.1):
    left_input = tf.keras.Input(shape=input_shape, name="left_hand_input")
    right_input = tf.keras.Input(shape=input_shape, name="right_hand_input")

    # Left hand BiLSTM
    left_bilstm_output = layers.Bidirectional(
        layers.LSTM(bilstm_unit_size, return_sequences=True),
        name="left_bilstm"
    )(left_input)  # shape: (30, 2*bilstm_unit_size)

    # Project to (30,63)
    left_projected = layers.Dense(63, activation='linear', name="left_project")(left_bilstm_output)

    # Right hand BiLSTM
    right_bilstm_output = layers.Bidirectional(
        layers.LSTM(bilstm_unit_size, return_sequences=True),
        name="right_bilstm"
    )(right_input)  # shape: (30, 2*bilstm_unit_size)

    # Project to (30,63)
    right_projected = layers.Dense(63, activation='linear', name="right_project")(right_bilstm_output)

    # Concatenate: (30,63) + (30,63) = (30,126)
    concatenated = layers.Concatenate(name="concat")([left_projected, right_projected])

    # Flatten (30,126) -> (3780,)
    x = layers.Flatten(name="flatten")(concatenated)

    # MLP layers
    for idx, units in enumerate(mlp_units):
        x = layers.Dense(units, activation='gelu', name=f"mlp_dense_{idx}")(x)
        x = layers.Dropout(dropout, name=f"mlp_dropout_{idx}")(x)

    # Output layer for classification
    output = layers.Dense(9, activation='softmax', name="classification_output")(x)

    model = tf.keras.Model(inputs=[left_input, right_input], outputs=output, name="classification_model")
    return model

X = np.load('NEW_DATASET/dataset_20240709_X_3096.npy')
y = np.load('NEW_DATASET/dataset_20240709_y_3096.npy')

data_folder = 'processed_datasets'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

split_60 = int(0.60 * len(X))
split_80 = int(0.80 * len(X))

(X_subject1, y_subject1), (X_subject1_aug, y_subject1_aug) = load_or_create_subject_datasets(
    X, y, subject_number=1, split_indices=(0, split_60), data_folder=data_folder)
(X_subject2, y_subject2), (X_subject2_aug, y_subject2_aug) = load_or_create_subject_datasets(
    X, y, subject_number=2, split_indices=(split_60, split_80), data_folder=data_folder)
(X_subject3, y_subject3), (X_subject3_aug, y_subject3_aug) = load_or_create_subject_datasets(
    X, y, subject_number=3, split_indices=(split_80, len(X)), data_folder=data_folder)

subject_datasets = {
    1: {"aug": (X_subject1_aug, y_subject1_aug), "original": (X_subject1, y_subject1)},
    2: {"aug": (X_subject2_aug, y_subject2_aug), "original": (X_subject2, y_subject2)},
    3: {"aug": (X_subject3_aug, y_subject3_aug), "original": (X_subject3, y_subject3)}
}

bilstm_unit_sizes = [32, 64, 128]  # You can pick any size now
mlp_units = [128,64]
dropouts = [0.1, 0.2]
learning_rates = [2e-4, 5e-4, 8e-4]
batch_size = 1024
epochs = 200

combinations_subjects = [
    {'train_subjects': [1, 2], 'test_subject': 3},
    {'train_subjects': [1, 3], 'test_subject': 2},
    {'train_subjects': [2, 3], 'test_subject': 1}
]

for bilstm_unit_size in bilstm_unit_sizes:
    for dropout in dropouts:
        for learning_rate in learning_rates:
            print("\n======================================")
            print(f"Testing hyperparameters: bilstm_unit_size={bilstm_unit_size}, dropout={dropout}, lr={learning_rate}, batch_size={batch_size}, epochs={epochs}")
            print("======================================\n")

            test_losses = []
            test_accuracies = []
            f1_scores = []

            for idx, combo in enumerate(combinations_subjects):
                print(f"\nRunning combination {idx+1}: Train on subjects {combo['train_subjects']}, Test on subject {combo['test_subject']}")

                train_subjects = combo['train_subjects']
                test_subject = combo['test_subject']

                X_train_hand1, X_train_hand2, y_train, \
                X_val_hand1, X_val_hand2, y_val, \
                X_test_hand1, X_test_hand2, y_test = prepare_datasets(train_subjects, test_subject)

                classification_model = build_classification_model(
                    bilstm_unit_size=bilstm_unit_size,
                    mlp_units=mlp_units,
                    dropout=dropout
                )

                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0)
                classification_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

                def lr_schedule(epoch):
                    warmup_epochs = 20
                    total_epochs = epochs
                    initial_lr = learning_rate
                    min_lr = 1e-8

                    if epoch < warmup_epochs:
                        lr = initial_lr * (epoch + 1) / warmup_epochs
                    else:
                        decay_epochs = total_epochs - warmup_epochs
                        lr = initial_lr - (initial_lr - min_lr) * (epoch - warmup_epochs + 1) / decay_epochs
                        lr = max(lr, min_lr)
                    return lr

                from tensorflow.keras.callbacks import LearningRateScheduler
                callbacks = [LearningRateScheduler(lr_schedule, verbose=0)]

                train_dataset = tf.data.Dataset.from_tensor_slices(
                    ((X_train_hand1, X_train_hand2), y_train)
                ).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                val_dataset = tf.data.Dataset.from_tensor_slices(
                    ((X_val_hand1, X_val_hand2), y_val)
                ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                classification_model.fit(
                    train_dataset,
                    epochs=epochs,
                    validation_data=val_dataset,
                    callbacks=callbacks,
                    verbose=0
                )

                test_dataset = tf.data.Dataset.from_tensor_slices(
                    ((X_test_hand1, X_test_hand2), y_test)
                ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

                y_pred_probs = classification_model.predict(test_dataset)
                y_pred = np.argmax(y_pred_probs, axis=1)
                y_true = np.argmax(y_test, axis=1)

                f1 = f1_score(y_true, y_pred, average='weighted')
                test_loss, test_accuracy = classification_model.evaluate(test_dataset, verbose=0)
                print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, F1 Score: {f1:.4f}')

                test_losses.append(test_loss)
                test_accuracies.append(test_accuracy)
                f1_scores.append(f1)

            # Average metrics over three subject combinations
            avg_test_loss = np.mean(test_losses)
            avg_test_accuracy = np.mean(test_accuracies)
            avg_f1_score = np.mean(f1_scores)

            print("\n*** AVERAGED RESULTS FOR THIS HYPERPARAMETER SET ***")
            print(f'Average Test Loss: {avg_test_loss:.4f}')
            print(f'Average Test Accuracy: {avg_test_accuracy:.4f}')
            print(f'Average F1 Score: {avg_f1_score:.4f}')
            print("****************************************************\n")
